In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-18 02:25:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.73MB/s    in 0.5s    

2021-11-18 02:25:33 (1.73 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

software_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), inferSchema=True, sep='\t')
software_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [ ]:
#Clean and Count
print(software_df.count())
software_df=software_df.dropna()
software_df=software_df.dropDuplicates()
print(software_df.count())

341931
341913


In [ ]:
#Examine Schema
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#Create software review table
software_review_table=software_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
software_review_table.show(5)

In [ ]:
#Create software product table
software_product_table=software_df.select(['product_id','product_title'])
software_product_table.show(5)

In [ ]:
#Create software customer table
customer_df = software_df.groupBy('customer_id').count()
customer_df.show(5)

In [ ]:
#Create new DF for vine
vine_reviews=software_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_reviews.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3UTVKJF44E0R6|          1|           46|         69|   N|
|R16IFUMCFXYVR2|          5|            0|          0|   N|
|R21XYG71U6C9IM|          5|            0|          0|   N|
| RSF2WJUKZ7GP4|          5|            0|          0|   N|
| R4OXOE6JIRQJN|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
#Taking Vine values with Y
vine_reviews_y = vine_reviews.filter(vine_reviews['vine']== 'Y')
vine_reviews_y.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R31Y4SHLQD7UU0|          1|            7|         13|   Y|
| RDMY8U7G00B1D|          4|            2|          2|   Y|
|R12HML0FDHA2UV|          4|            0|          1|   Y|
|R32BYGORDU46YZ|          4|            0|          0|   Y|
| RJZ0MGZNQ9T0E|          4|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
#Write DF to RDS
mode="append"
jdbc_url = "jdbc:postgresql://<your connection string>:5432/<your database-name>"
config = {"user":"<your database user>",
          "password": "<your database password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
vine_reviews_y.write.jdbc(url=jdbc_url, table='software_vine_reviews', mode=mode, properties=config)

In [ ]:
software_review_table.write.jdbc(url=jdbc_url, table='software_review_id_table', mode=mode, properties=config)

In [ ]:
software_product_table.write.jdbc(url=jdbc_url, table='software_products', mode=mode, properties=config)

In [ ]:
customer_df.write.jdbc(url=jdbc_url, table='software_customers', mode=mode, properties=config)